# 10-2 Treasury Bond Fund Simulator

This is based off of [longinvest's post on bogleheads](1). He implemented it all in a spreadsheet (which is linked in the thread).

The goal is to calculate returns of a simulated bond fund given a bunch of interest rates.

First we need to import some libraries....

In [1]:
import numpy
from collections import deque
import pandas
import math

In [2]:
def iterate_fund(ladder, yield_curve, steps=9):
    payments = get_payments(ladder)

    sold_bond = ladder.popleft()
    payments += sold_bond.sell(yield_curve[0])

    new_bond = Bond(payments, yield_curve[-1], steps+1)
    ladder.append(new_bond)
    
    # This happens *after* we sell the shortest bond and buy a new long one
    # (at least, that's what longinvest does...)
    nav = get_nav(ladder, yield_curve)

    return (ladder, payments, nav)

We have a number of sources of rate data.

* Shiller provides 10 year yields on Treasuries, going back to 1871
* Shiller provides 1 year interest rates, going back to 1871
* FRED provides 1-, 2-, 3-, 5-, and 7-year rates. The data begins in the 1954-1977 range. When available, we prefer the FRED data over Shiller data.


In [3]:
df = pandas.read_csv('bond_rates.csv')
df.head()

,Year,1 year,2 year,3 year,5 year,7 year,10 year
0,1871,0.0635,NaN,NaN,NaN,NaN,0.0532
1,1872,0.0781,NaN,NaN,NaN,NaN,0.0536
2,1873,0.0835,NaN,NaN,NaN,NaN,0.0558
3,1874,0.0686,NaN,NaN,NaN,NaN,0.0547
4,1875,0.0496,NaN,NaN,NaN,NaN,0.0507


For a given year, we will have some rate data. At the very least we will have the 1-year and 10-year rates; the data on those go back the further.

However, we may *also* have other rate data from FRED.

But we need to have rate data for every year on the yield curve. That is: 1-, 2-, 3-, 4-, 5-, 6-, 7-, 9-, and 10-year rates. When we don't have the data available we will perform linear interpolation from data we *do* have to fill in the gaps.

So if we only have the 1- and 10-year data then we need to do a linear interpolation for the other 8 years. If we have 1-, 3-, and 10-year data then we do linear interpolation between the 1- and 3-year data to fill in the 2-year data. And we'll do linear interpolation between the 3- and 10-year data for the rest.

In [4]:
def interpolate_rates(raw_rates, steps=9):
    # First try to pre-load any FRED rates.
    # We use NaN to indicate "the data needs to be interpolated"
    s = pandas.Series(math.nan, index=numpy.arange(steps+1))
    s.iloc[0] = raw_rates['1 year']
    s.iloc[1] = raw_rates['2 year']
    s.iloc[2] = raw_rates['3 year']
    s.iloc[4] = raw_rates['5 year']
    s.iloc[6] = raw_rates['7 year']
    s.iloc[9] = raw_rates['10 year']
    
    def left_number(series, index):
        if not math.isnan(series.iloc[index]):
            return index
        else:
            return left_number(series, index-1)
        
    def right_number(series, index):
        if not math.isnan(series.iloc[index]):
            return index
        else:
            return right_number(series, index+1)
        
    # now fill in the gaps with linear interpolation.
    for i in (1, 2, 3, 4, 5, 6, 7, 8):
        if math.isnan(s.iloc[i]):
            #print('Interpolating year', i+1)
            left = left_number(s, i)
            right = right_number(s, i)
            steps = right - left
            # Once we've figured out where there is good data around the hole,
            # we can interpolate.
            rate = s.iloc[left] + ((s.iloc[right] - s.iloc[left]) * (i - left) / steps)
            s.iloc[i] = rate

    return s.tolist()

In [5]:
interpolate_rates(df.iloc[0])

[0.0635,
 0.06235555555555555,
 0.061211111111111105,
 0.060066666666666664,
 0.05892222222222222,
 0.057777777777777775,
 0.05663333333333333,
 0.055488888888888886,
 0.054344444444444445,
 0.0532]




A ladder is built from 10-year maturity down as short as you want.

We need the current interest rate to determine the yield on the shortest maturity we hold :(

longinvest uses data from Shiller up to 1954 and then uses FRED 1 year Treasury data after that.

We need monthly data....

* FRED has 1-month Treasury data at https://fred.stlouisfed.org/series/GS1M but it only goes back to 2001
* FRED has 3-month at https://fred.stlouisfed.org/series/DGS3MO going back to 1982
* FRED has 1-year (monthly data) going back to 1953 at https://fred.stlouisfed.org/series/GS1

longinvest builds a 10-2 ladder using 9 bonds: 10-year maturity, 9-year maturity, etc down to 2-year maturity

Except there's a bootstrap period in the first decade...

So let's try building a similar ladder but with monthly granularity. That means instead of 9 bonds we want (12 * 9) + 11 = 119 bonds. Every month we sell a bond that is about to mature (that has 1 month left) and buy a brand new 10 year bond at the current rate.

Let's start in July 2001 so we can use the FRED monthly data.

The starting FRED rate is 3.67
Shiller's 10-year yield is 5.24

[1]: https://www.bogleheads.org/forum/viewtopic.php?f=10&t=179425

In [6]:
class Bond:
    def __init__(self, face_value, yield_pct, maturity):
        self.face_value = face_value
        self.yield_pct = yield_pct
        self.maturity = maturity
        
    def __repr__(self):
        return ('M: %d | Y: %.2f%% | FV: $%.2f' % (self.maturity, self.yield_pct * 100, self.face_value))
        
    def gen_payment(self):
        return self.face_value * self.yield_pct
    
    def sell(self, current_rate):        
        pv = -numpy.pv(current_rate, 1, self.gen_payment(), self.face_value)
        return pv
    
    def value(self, rates):
        value = numpy.pv(rates[self.maturity - 1], self.maturity, self.face_value * self.yield_pct, self.face_value)
        return -value

In [7]:
def get_nav(ladder, rates):
    return sum((b.value(rates) for b in ladder))

def get_payments(ladder):
    return sum((b.gen_payment() for b in ladder))

def reduce_maturity(ladder):
    for b in ladder:
        b.maturity -= 1
    return ladder

def bootstrap(long_yield, steps=9):
    ladder = deque()
    starting_face_value = 50 # chosen arbitrarily (to match longinvest)
    for i in range(steps):
        face_value = pow(1 + long_yield, i) * starting_face_value
        b = Bond(face_value, long_yield, i+1)
        ladder.append(b)
    return ladder

def loop(bootstrap_rates, rates):
    ladder = bootstrap(bootstrap_rates['10 year'], steps=9)
    for (_, current_rates) in rates:
        (ladder, payments, nav) = iterate_fund(ladder, interpolate_rates(current_rates))
        print("%.2f, %.2f" % (nav, payments))
        reduce_maturity(ladder)

In [8]:
loop(df.iloc[0], df.iterrows())

578.50, 79.24
593.91, 82.72
615.71, 86.82
666.30, 92.37
731.56, 98.48
773.55, 103.25
815.55, 108.19
855.72, 113.10
905.13, 118.50
934.70, 122.40
984.79, 126.88
1014.16, 131.05
1047.44, 136.34
1078.52, 141.70
1146.66, 147.52
1190.22, 152.74
1185.63, 155.90
1242.96, 162.86
1302.32, 167.87
1329.73, 171.93
1352.19, 176.44
1449.90, 185.89
1381.69, 186.99
1564.17, 201.37
1639.67, 208.98
1613.45, 210.51
1746.70, 223.22
1804.94, 229.33
1884.09, 235.53
1895.83, 239.65
1967.32, 250.73
2006.70, 252.42
2032.97, 266.12
2133.63, 276.73
2206.24, 281.05
2233.06, 292.74
2260.58, 299.45
2362.97, 310.22
2531.55, 322.85
2543.88, 334.53
2693.89, 344.69
2781.81, 363.39
2796.05, 376.57
2991.13, 391.02
3161.37, 411.68
3314.28, 426.52
3397.35, 442.36
3407.37, 456.06
3590.88, 477.98
3597.34, 491.13
3749.18, 519.58
4210.19, 555.78
4355.12, 574.85
4627.06, 606.90
4878.47, 630.06
5055.27, 651.99
5288.24, 672.54
5425.52, 694.09
5428.43, 704.66
5845.27, 743.58
6211.86, 783.25
6261.72, 800.00
6785.11, 846.41
7115.99,

It matches the 10-2 spreadsheet perfectly up until 1954...at which point longinvest starts using FRED data where available (e.g. for 1-year, 3-year, and 5-year) instead of continuing with linear interpolation.

TODO

- First, generalise things. Make it so I can do 10-2, 10-4, 4-2, 3-2, 30-11, 20-2.
- Handle the 2-year fund. What's this?
- Handle "Bond Ladder". The "Bond Fund" sells prior to maturity. The "Bond Ladder" holds to maturity.
- Once everything works for annual data, try monthly.